---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

In [2]:
def petaToGiga(row):
    row['Energy Supply'] *= 1000000
    return row

def getData():
    #Reading and parsing energy ('Energy Indicators.xls')
    energy = pd.read_excel('Energy Indicators.xls', header=None, skip_footer=2)
    #print (energy)
    energy = (energy.drop([0, 1], axis=1) #Remove first two columns
                     .dropna() #drop NaN's
                     .drop(9)
                     .rename(columns={2: 'Country', #Renaming columns
                                      3: 'Energy Supply',
                                      4: 'Energy Supply per Capita',
                                      5: '% Renewable'})
                     .replace(regex=True, to_replace=[r'\d', r' \(([^)]+)\)'], value=r'') #Pruning in country names
                     .replace(to_replace=["...", "Republic of Korea", "United States of America", #Replace ... with NaN and country names
                                         "United Kingdom of Great Britain and Northern Ireland",
                                         "China, Hong Kong Special Administrative Region"],
                              value=[np.NaN, "South Korea", "United States", "United Kingdom", "Hong Kong"])
                     .apply(petaToGiga, axis=1,)) #convert petajoules to giga joules
    #print (energy)
    #Reading GDP ('world_bank.csv')
    GDP = pd.read_csv('world_bank.csv', header=None, skiprows=4)
    GDP = (GDP.rename(columns=GDP.iloc[0])
              .drop(0)
              .replace(to_replace=["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"],
                       value=["South Korea", "Iran", "Hong Kong"])
              .rename(columns={2006: '2006', 2007: '2007', 2008: '2008', 2009: '2009', 2010: '2010',
                                    2011: '2011', 2012: '2012', 2013: '2013', 2014: '2014', 2015: '2015'}))
    #Reading ScimEn('scimagojr-3.xlsx')
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    return energy, GDP, ScimEn

def answer_one():
    energy, GDP, ScimEn = getData()
    energy = energy.dropna()
    GDP_columns = ['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    GDP = GDP[GDP_columns] #Prune GDP
    ScimEn_columns = ['Rank', 'Country', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index']
    ScimEn = ScimEn[ScimEn_columns]
    ScimEn = ScimEn[:15] #Prune ScimEn
    #Merging and cleaning dataframes
    new = pd.merge(energy, GDP, how="inner", left_on="Country", right_on="Country Name")
    new = new.drop(['Country Name'], axis=1)
    new = pd.merge(new, ScimEn, how="inner", left_on='Country', right_on='Country')
    new = new.set_index('Country')
    columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    new = new[columns]
    return new
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5.386000e+09,231.0,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119.0,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109.0,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [3]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [4]:
def answer_two():
    energy, GDP, ScimEn = getData()
    m1 = pd.merge(energy, GDP, how="inner", left_on="Country", right_on="Country Name")
    m2 = pd.merge(energy, ScimEn, how="inner", left_on="Country", right_on="Country")
    m3 = pd.merge(GDP, ScimEn, how="inner", left_on="Country Name", right_on="Country")
    actual = pd.merge(m1, ScimEn, how="inner", left_on="Country", right_on="Country")
    g = len(energy.index)
    r = len(GDP.index)
    p = len(ScimEn.index)
    top = len(m1.index)
    left = len(m2.index)
    right = len(m3.index)
    all3 = len(actual.index)
    ans = g + r + p - top - left - right + all3
    return ans
answer_two()

318

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP **sorted** in descending order.

In [5]:
def answer_three():
    finalMERGE = answer_one()
    finalMERGE['avgGDP'] = finalMERGE.iloc[:, 10:].mean(axis=1)
    return finalMERGE['avgGDP'].sort_values(ascending=False)
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [6]:
# finalMERGE['avgGDP'].sort_values()

In [7]:
# finalMERGE.iloc[3,19] - finalMERGE.iloc[3,10]

In [8]:
# finalMERGE.loc[:, '2015'] - finalMERGE.loc[:, '2006']

In [9]:
# (finalMERGE.loc[:, '2015'] - finalMERGE.loc[:, '2006']).values[3]

In [10]:
def answer_four():
    Top15 = answer_one()
    avgGDP = answer_three()
    country = avgGDP.keys()[5]
    ans = Top15.loc[country, '2015'] - Top15.loc[country, '2006']
    return ans
answer_four()

246702696075.3999

In [11]:
# def answer_four():
#     Top15 = answer_one()
#     Top15 = Top15.reset_index()
#     return Top15.iloc[3,19] - Top15.iloc[3,10]

# answer_four()

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [12]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [13]:
def answer_six():
    finalMERGE = answer_one()
    return finalMERGE['% Renewable'].argmax(),finalMERGE.loc[finalMERGE['% Renewable'].argmax(), '% Renewable'] 
answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [14]:
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5.386000e+09,231.0,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119.0,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109.0,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12


In [15]:
def answer_seven():
    finalMERGE = answer_one()
    finalMERGE['Ratio'] = finalMERGE['Self-citations'] / finalMERGE['Citations']
    return finalMERGE['Ratio'].argmax(), finalMERGE['Ratio'].sort_values(ascending=False)[0]

answer_seven()

('China', 0.68931261793894216)

In [16]:
# def ratio(row):
#     row['ratio'] = row['Self-citations'] / row['Citations']
#     return row

# def answer_seven():
#     Top15 = answer_one()
#     new = Top15.apply(ratio, axis=1)
#     m = float(np.max(new['ratio']))
#     country = new[new['ratio'] == m].iloc[0].name
#     return country, m
# answer_seven()

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [17]:
# (finalMERGE.iloc[:,7] / finalMERGE.iloc[:,8]).sort_values()

In [18]:
def answer_eight():
    finalMERGE = answer_one()
    finalMERGE['population'] = finalMERGE['Energy Supply'] / finalMERGE['Energy Supply per Capita']
    return finalMERGE['population'].sort_values(ascending=False).index[2]

answer_eight()

'United States'

In [19]:
# def answer_eight():
#     Top15 = answer_one()
#     new = Top15.apply(lambda x: x['Energy Supply'] / x['Energy Supply per Capita'], axis=1)
#     new.sort(ascending=False)
#     country = new.keys()[2]
#     return country
# answer_eight()

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [20]:
# finalMERGE['Citable Documents Per Person'] = finalMERGE['Citable documents'] / finalMERGE['population']
# finalMERGE['Citable Documents Per Person'].corr(finalMERGE['Energy Supply per Capita'])

In [21]:
def answer_nine():
    df = answer_one()
    df['population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df['citable documents per person'] = df['Citable documents'] / df['population']
    return df['citable documents per person'].corr(df['Energy Supply per Capita'])

answer_nine()

0.79400104354429446

In [22]:
# def answer_nine():
#     Top15 = answer_one().copy()
#     Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
#     Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
#     Top15 = Top15.corr(method='pearson')
#     ans = Top15.loc['Energy Supply per Capita', 'Citable docs per Capita']
#     return ans
# answer_nine()

In [23]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [24]:
# plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [25]:
# # MINE
# def answer_ten():
#     finalMERGE = answer_one()
    
#     def fun10(x):
#         if x > finalMERGE['% Renewable'].median():
#             return 1
#         else:
#             return 0

#     finalMERGE['HighRenew'] = finalMERGE['% Renewable'].apply(fun10)
#     return finalMERGE['HighRenew'].sort_values()
# answer_ten()

In [26]:
def answer_ten():
    Top15 = answer_one()
    median = np.median(Top15['% Renewable'])
    for i in range(len(Top15)):
        if Top15.iloc[i]['% Renewable'] >= median:
            Top15.set_value(Top15.iloc[i].name, 'HighRenew', 1)
        else:
            Top15.set_value(Top15.iloc[i].name, 'HighRenew', 0)
    Top15 = Top15['HighRenew']
    Top15 = Top15.sort(inplace=False)
    return Top15
answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


Country
Australia             0.0
India                 0.0
Iran                  0.0
Japan                 0.0
South Korea           0.0
United Kingdom        0.0
United States         0.0
Brazil                1.0
Canada                1.0
China                 1.0
France                1.0
Germany               1.0
Italy                 1.0
Russian Federation    1.0
Spain                 1.0
Name: HighRenew, dtype: float64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [27]:
def answer_eleven():
    df = answer_one()
    df['population']=df['Energy Supply']/df['Energy Supply per Capita']

    ContinentDict = {'China':'Asia', 'United States':'North America', 'Japan':'Asia', 'United Kingdom':'Europe',
                     'Russian Federation':'Europe', 'Canada':'North America',
                     'Germany':'Europe','India':'Asia','France':'Europe','South Korea':'Asia',
                     'Italy':'Europe','Spain':'Europe','Iran':'Asia','Australia':'Australia','Brazil':'South America'}
    df.rename(index=ContinentDict,inplace=True)
    df.reset_index(inplace=True)
    functions = ['size', 'sum', 'mean', 'std']
    result = df[['Country','population']].groupby('Country').agg(functions)
    return (result)

answer_eleven()

population                                          
                    size           sum          mean           std
Country                                                           
Asia                   5  2.898666e+09  5.797333e+08  6.790979e+08
Australia              1  2.331602e+07  2.331602e+07           NaN
Europe                 6  4.579297e+08  7.632161e+07  3.464767e+07
North America          2  3.528552e+08  1.764276e+08  1.996696e+08
South America          1  2.059153e+08  2.059153e+08           NaN

In [28]:
# ContinentDict  = {'China':'Asia',
#                   'United States':'North America',
#                   'Japan':'Asia',
#                   'United Kingdom':'Europe',
#                   'Russian Federation':'Europe',
#                   'Canada':'North America',
#                   'Germany':'Europe',
#                   'India':'Asia',
#                   'France':'Europe',
#                   'South Korea':'Asia',
#                   'Italy':'Europe',
#                   'Spain':'Europe',
#                   'Iran':'Asia',
#                   'Australia':'Australia',
#                   'Brazil':'South America'}

# def answer_eleven():
#     Top15 = answer_one()
#     Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
#     for i in range(len(Top15)):
#         country = Top15.iloc[i].name
#         Top15.set_value(country, 'Continent', ContinentDict[country])
#     Top15 = (Top15.reset_index(level=0)
#                  .set_index(['Continent', 'Country']))
#     Top15 = Top15.groupby(level=0)['PopEst'].agg({'size': np.size, 'sum': np.sum, 'mean': np.average, 'std': np.std})
#     return Top15
# answer_eleven()

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [29]:
def answer_twelve():
    df=answer_one()
    df['bin'] = pd.cut(df['% Renewable'],5)
    ContinentDict = {'China':'Asia', 'United States':'North America', 'Japan':'Asia', 'United Kingdom':'Europe', 'Russian Federation':'Europe', 'Canada':'North America',
                     'Germany':'Europe',
                     'India':'Asia',
                     'France':'Europe',
                     'South Korea':'Asia',
                     'Italy':'Europe',
                     'Spain':'Europe',
                     'Iran':'Asia',
                     'Australia':'Australia',
                     'Brazil':'South America'}  
    df.rename(index=ContinentDict,inplace= True) 
    df.reset_index(inplace = True)
    return df.groupby(['Country','bin']).size()

answer_twelve()

Country        bin             
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [30]:
def answer_thirteen():
    finalMERGE = answer_one()
    finalMERGE['population'] = finalMERGE['Energy Supply'] / finalMERGE['Energy Supply per Capita']
    finalMERGE['PopEst'] = finalMERGE['population'].apply(lambda x: format(x,','))
    return finalMERGE['PopEst']
answer_thirteen()

Country
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
France                  63,837,349.39759036
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
Iran                    77,075,630.25210084
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
South Korea            49,805,429.864253394
Russian Federation            143,500,000.0
Spain                    46,443,396.2264151
United Kingdom         63,870,967.741935484
United States          317,615,384.61538464
Name: PopEst, dtype: object

In [31]:
# def answer_thirteen():
#     Top15 = answer_one()
#     Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
#     Top15 = Top15['PopEst']
#     for i in range(len(Top15)):
#         country = Top15.keys()[i]
#         number = "{:,}".format((Top15.iloc[i]))
#         Top15.replace(Top15.iloc[i], number, inplace=True)
#     return Top15
# answer_thirteen()

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [32]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [33]:
# plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!